In [1]:
import pandas as pd
import datetime

In [2]:
est_data = pd.read_csv("C:/model_data/estimate_raw.csv")

In [3]:
est_data.drop('Unnamed: 0', inplace = True, axis = 1)

In [4]:
# source_id and currency -> category for 
est_data['source_id'] = est_data['source_id'].astype('category')
est_data['currency'] = est_data['currency'].astype('category')

In [5]:
# now we need to convert estimate and period dates to dates
est_data['period_date'] = pd.to_datetime(est_data['period_date'])
est_data['estimate_date'] = pd.to_datetime(est_data['estimate_date'])

In [6]:
#managed to reduce the mem usage by a third

In [7]:
lag_tol = 90 # 90 days is the minimum lag tolerance

max_est = (est_data
               .groupby(['security_id', 'broker_id', 'period_date'])
               ['estimate_date']
               .max()
          )

min_est = (est_data
               .groupby(['security_id', 'broker_id', 'period_date'])
               ['estimate_date']
               .min()
          )

lag_bool = (max_est - min_est).dt.days > lag_tol

est_data = (
    est_data
    .join(lag_bool, 
          on = ['security_id', 'broker_id', 'period_date'],
          rsuffix = '_in')
)

In [8]:
est_data = est_data[est_data.estimate_date_in == True]
est_data.drop('estimate_date_in', axis = 1, inplace = True)

In [9]:
est_data.sort_values(['security_id', 'broker_id', 'period_date', 'estimate_date'], inplace = True)

In [118]:
# This just makes the time series daily between revision dates
est_data_daily = (
    est_data.head(15)
    .set_index(['security_id', 'broker_id', 'period_date'])
    .groupby(['security_id', 'broker_id', 'period_date'])
    .apply(
        lambda df: df.drop_duplicates('estimate_date')
                     .set_index('estimate_date')
                     .resample('D')
                     .ffill()
    )
)

In [128]:
def shift_df(df, lag_tol):
    # shift data
    shifted_df = df.shift(periods = -lag_tol)
    # assign na
    shifted_df.value.fillna(est_data_daily.value[-1], inplace = True)
    shifted_df.currency.fillna(est_data_daily.currency[-1], inplace = True)
    shifted_df.source_id.fillna(est_data_daily.source_id[-1], inplace = True)
    # shift date
    shifted_df.reset_index(['security_id', 'broker_id', 'period_date', 'estimate_date'], 
                           inplace = True)
    shifted_df['estimate_date'] = est_daily_shift['estimate_date'] + datetime.timedelta(days = 90)
    return shifted_df

In [129]:
for name_gp, df_gp in est_data_daily.groupby(['security_id', 'broker_id', 'period_date']):
    df_shift = shift_df(df_gp, lag_tol)
# something is going wrong with the date shifting...
df_shift

,security_id,broker_id,period_date,estimate_date,value,currency,source_id
0,30064771087,-1906764500,2020-08-31,2020-02-16,7.70,USD,ibes
1,30064771087,-1906764500,2020-08-31,2020-02-17,7.72,USD,ibes
2,30064771087,-1906764500,2020-08-31,2020-02-18,7.72,USD,ibes
3,30064771087,-1906764500,2020-08-31,2020-02-19,7.72,USD,ibes
4,30064771087,-1906764500,2020-08-31,2020-02-20,7.72,USD,ibes
...,...,...,...,...,...,...,...
262,30064771087,-1906764500,2020-08-31,NaT,7.80,USD,ibes
263,30064771087,-1906764500,2020-08-31,NaT,7.80,USD,ibes
264,30064771087,-1906764500,2020-08-31,NaT,7.80,USD,ibes
265,30064771087,-1906764500,2020-08-31,NaT,7.80,USD,ibes


In [11]:
# We still need a lookahead function that determines whether there is an upward or downward 
# revision in the next 90 days, and then to shift the estimate date 90 days forward
# on a copied data frame that can be outer joined back to the result
# for a shifted timeseries
est_daily_shift = est_data_daily.shift(periods = -90)

# filling in NAs
est_daily_shift.value.fillna(est_data_daily.value[-1], inplace = True)
est_daily_shift.currency.fillna(est_data_daily.currency[-1], inplace = True)
est_daily_shift.source_id.fillna(est_data_daily.source_id[-1], inplace = True)

In [12]:
# now we need to shift the estimate date column only by ninety days into the future
est_daily_shift.reset_index(['security_id', 'broker_id', 'period_date', 'estimate_date'], inplace = True)
est_daily_shift['estimate_date'] = (
    est_daily_shift['estimate_date'] 
    + datetime.timedelta(days = 90)
)

In [13]:
est_data_daily.reset_index(['security_id', 'broker_id', 'period_date', 'estimate_date'], inplace = True)

In [71]:
# now we need the initial portion of est_data_daily...
df_append = est_data_daily[est_data_daily.estimate_date < est_daily_shift.estimate_date.min()]
est_data_slide = (pd.concat([df_append, est_daily_shift])
                      .set_index(['security_id', 'broker_id', 'period_date', 'estimate_date'])
                 )

In [72]:
est_data_slide.reset_index('estimate_date', inplace = True)

In [73]:
# now we can calculate the differences and assign them into a timeseries
changes = (est_data_slide['value'][lag_tol:]
               .subtract(est_data_slide['value'][:(-lag_tol)])
          )
changes.reset_index(drop = True, inplace = True)

In [74]:
est_data_slide.reset_index(['security_id', 'broker_id', 'period_date'], inplace = True)

In [94]:
changes_lag = (changes.shift(lag_tol, fill_value = 0)[:lag_tol]
                   .append(changes)
                   .rename('value_changes')
              )
changes_lag.reset_index(drop = True, inplace = True)

In [95]:
est_data_slide.insert(6, 'changes', changes_lag, allow_duplicates = True)

In [102]:
est_data_slide = est_data_slide.assign(up = (est_data_slide.changes > 0))
est_data_slide = est_data_slide.assign(down = (est_data_slide.changes < 0))

In [103]:
est_data_slide
# now I just have to regroup and perform the for loop

,security_id,broker_id,period_date,estimate_date,value,currency,changes,source_id,up,down
0,30064771087,-2084193872,2020-08-31,2019-08-20,8.2,USD,0.00,ibes,False,False
1,30064771087,-2084193872,2020-08-31,2019-08-21,8.2,USD,0.00,ibes,False,False
2,30064771087,-2084193872,2020-08-31,2019-08-22,8.2,USD,0.00,ibes,False,False
3,30064771087,-2084193872,2020-08-31,2019-08-23,8.2,USD,0.00,ibes,False,False
4,30064771087,-2084193872,2020-08-31,2019-08-24,8.2,USD,0.00,ibes,False,False
...,...,...,...,...,...,...,...,...,...,...
298,30064771087,-2084193872,2020-08-31,2020-06-13,7.7,USD,-0.14,ibes,False,True
299,30064771087,-2084193872,2020-08-31,2020-06-14,7.7,USD,-0.14,ibes,False,True
300,30064771087,-2084193872,2020-08-31,2020-06-15,7.7,USD,-0.14,ibes,False,True
301,30064771087,-2084193872,2020-08-31,2020-06-16,7.7,USD,-0.14,ibes,False,True
